In [ ]:
import requests

def query_prometheus(prometheus_url, query):
    url = f"{prometheus_url}/api/v1/query"
    
    params = {
        'query': query
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to query Prometheus: {response.status_code}, {response.text}")

prometheus_url = "http://localhost:9090"
query = 'node_cpu_seconds_total'
metrics_data = query_prometheus(prometheus_url, query)
print(metrics_data)


In [ ]:
def process_metrics_data(data):
    result = data['data']['result']
    processed_data = {}
    
    for metric in result:
        metric_name = metric['metric'].get('__name__', 'unknown_metric')
        value = float(metric['value'][1])
        
        if metric_name in processed_data:
            processed_data[metric_name].append(value)
        else:
            processed_data[metric_name] = [value]
    
    for metric_name, values in processed_data.items():
        processed_data[metric_name] = sum(values) / len(values)
    
    return processed_data

processed_data = process_metrics_data(metrics_data)
print(processed_data)


In [ ]:
def send_data_to_dotnet_web(api_url, processed_data):
    response = requests.post(api_url, json=processed_data)
    
    if response.status_code == 200:
        print("Data successfully sent to .NET web application.")
    else:
        raise Exception(f"Failed to send data: {response.status_code}, {response.text}")
processed_data = {
    "metricName": "CPU Usage",
    "value": 75,
    "timestamp": "2024-11-28T14:00:00"
}
api_url = "http://localhost:5001/api/monitoring/metrics"
send_data_to_dotnet_web(api_url, processed_data)


In [ ]:
# import requests

# def query_prometheus(prometheus_url, query):
#     url = f"{prometheus_url}/api/v1/query"
#     params = {'query': query}
#     response = requests.get(url, params=params)
    
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise Exception(f"Failed to query Prometheus: {response.status_code}, {response.text}")

# def process_metrics_data(data):
#     result = data['data']['result']
#     processed_data = {}
    
#     for metric in result:
#         metric_name = metric['metric'].get('__name__', 'unknown_metric')
#         value = float(metric['value'][1])
        
#         if metric_name in processed_data:
#             processed_data[metric_name].append(value)
#         else:
#             processed_data[metric_name] = [value]
    
#     for metric_name, values in processed_data.items():
#         processed_data[metric_name] = sum(values) / len(values)
    
#     return processed_data

# def send_data_to_dotnet_web(api_url, processed_data):
#     response = requests.post(api_url, json=processed_data)
    
#     if response.status_code == 200:
#         print("Data successfully sent to .NET web application.")
#     else:
#         raise Exception(f"Failed to send data: {response.status_code}, {response.text}")

# prometheus_url = "http://<PROMETHEUS_SERVER>:9090"
# query = 'node_cpu_seconds_total'
# metrics_data = query_prometheus(prometheus_url, query)
# processed_data = process_metrics_data(metrics_data)
# api_url = "http://<YOUR_DOTNET_APP>/api/monitoring/metrics"
# send_data_to_dotnet_web(api_url, processed_data)
